In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler

# Feature Engineering

In [30]:
data_dir = "finalData.csv"
df = pd.read_csv(data_dir)

# renaming the columns
df.columns = ['Date', 'Home Team', 'Away Team', 'Full Time Home Goals', 'Full Time Away Goals', 'Full Time Result',       
'Half Time Home Goals', 'Half Time Away Goals', 'Half Time Result', 'Referee', 'Home Shots', 'Away Shots', 'Home Shots on Target',   
'Away Shots on Target', 'Home Corners', 'Away Corners', 'Home Fouls', 'Away Fouls', 'Home Yellow Cards', 'Away Yellow Cards',     
'Home Red Cards', 'Away Red Cards', 'Home Possession', 'Away Possession', 'Home Passes Completed', 'Home Passes PCT',
'Home Progressive Passes', 'Home Progressive Passing Distance', 'Home XG', 'Home Take Ons Won', 'Home Take Ons', 
'Home Interceptions', 'Home Blocks', 'Home Touches', 'Home Touches Def 3rd', 'Home Touches Mid 3rd', 'Home Touches Att 3rd',
'Home Carries', 'Home Carries Progressive Distance', 'Home Tackles', 'Home Tackles Won', 'Away Passes Completed',
'Away Passes PCT', 'Away Progressive Passes', 'Away Progressive Passing Distance', 'Away XG', 'Away Shots REDUNDANT', 'Away Shots on Target REDUNDANT', 
'Away Take Ons Won', 'Away Take Ons', 'Away Interceptions', 'Away Blocks', 'Away Touches', 'Away Touches Def 3rd',
'Away Touches Mid 3rd', 'Away Touches Att 3rd', 'Away Carries', 'Away Carries Progressive Distance', 'Away Tackles',
'Away Tackles Won']

df.drop(columns=['Away Shots on Target REDUNDANT', 'Away Shots REDUNDANT'], inplace=True)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,home_possession,away_possession,home_passes_completed,home_passes_pct,home_progressive_passes,home_progressive_passing_distance,home_xg,home_take_ons_won,home_take_ons,home_interceptions,home_blocks,home_touches,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_carries,home_carries_progressive_distance,home_tackles,home_tackles_won,away_passes_completed,away_passes_pct,away_progressive_passes,away_progressive_passing_distance,away_xg,away_shots,away_shots_on_target,away_take_ons_won,away_take_ons,away_interceptions,away_blocks,away_touches,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_carries,away_carries_progressive_distance,away_tackles,away_tackles_won
0,2017-08-11,Arsenal,Leicester City,4.0,3.0,H,2.0,2.0,D,M Dean,27.0,6.0,10.0,3.0,9.0,4.0,9.0,12.0,0.0,1.0,0.0,0.0,68,32,565,83.0,62,3216,2.6,16,19,13,6,801,203,355,252,519,1686,23,17,192,61.0,16,2006,1.5,6,3,5,16,11,18,427,169,142,120,150,422,17,8
1,2017-08-12,Brighton & Hove Albion,Manchester City,0.0,2.0,A,0.0,0.0,D,M Oliver,6.0,14.0,2.0,4.0,3.0,10.0,6.0,9.0,0.0,2.0,0.0,0.0,23,77,146,58.9,5,1685,0.3,7,9,12,10,343,177,119,51,126,366,10,8,718,87.6,87,3285,1.9,14,4,6,12,9,9,897,130,474,298,566,1779,10,6
2,2017-08-12,Chelsea,Burnley,2.0,3.0,A,0.0,3.0,A,C Pawson,19.0,10.0,6.0,5.0,8.0,5.0,16.0,11.0,3.0,3.0,2.0,0.0,62,38,468,82.4,48,2448,1.5,5,8,9,6,667,201,268,203,394,1440,10,8,257,73.9,26,2521,0.6,10,5,2,3,13,13,453,168,212,74,190,460,8,4
3,2017-08-12,Crystal Palace,Huddersfield Town,0.0,3.0,A,0.0,2.0,A,J Moss,14.0,8.0,4.0,6.0,12.0,9.0,7.0,19.0,1.0,3.0,0.0,0.0,56,45,333,75.0,33,2847,1.1,18,23,19,9,561,180,274,113,238,751,24,19,230,64.6,18,2165,1.5,9,6,5,11,9,12,460,173,166,127,167,452,28,19
4,2017-08-12,Everton,Stoke City,1.0,0.0,H,1.0,0.0,H,N Swarbrick,9.0,9.0,4.0,1.0,6.0,7.0,13.0,10.0,1.0,1.0,0.0,0.0,60,40,425,78.0,32,2670,0.6,1,7,19,6,670,227,308,142,313,880,17,12,233,65.4,34,2068,0.4,9,1,10,13,24,10,473,154,192,135,190,729,18,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,2024-05-19,Crystal Palace,Aston Villa,5.0,0.0,H,2.0,0.0,H,D Bond,15.0,8.0,9.0,2.0,10.0,8.0,2.0,4.0,1.0,4.0,0.0,0.0,54,46,509,87.9,45,2499,2.5,6,12,3,12,663,122,348,199,396,969,17,13,428,85.8,11,2182,1.0,8,2,5,10,2,4,566,291,219,62,333,709,10,2
2756,2024-05-19,Liverpool,Wolverhampton Wanderers,2.0,0.0,H,2.0,0.0,H,C Kavanagh,36.0,4.0,14.0,3.0,14.0,11.0,10.0,2.0,1.0,1.0,0.0,1.0,67,33,592,89.2,76,3280,4.9,5,21,8,10,779,102,342,344,428,1314,22,13,250,75.3,15,1666,0.5,4,2,15,27,3,15,441,216,164,65,202,722,23,11
2757,2024-05-19,Luton Town,Fulham,2.0,4.0,A,1.0,2.0,A,M Donohue,15.0,15.0,6.0,7.0,15.0,20.0,4.0,4.0,5.0,4.0,0.0,0.0,41,59,265,76.6,27,2089,2.1,5,16,5,12,429,148,180,110,216,702,9,4,415,83.7,40,2849,1.1,16,8,5,12,4,9,590,235,229,129,337,650,20,9
2758,2024-05-19,Manchester City,West Ham United,3.0,1.0,H,2.0,1.0,H,J Brooks,28.0,3.0,12.0,2.0,3.0,12.0,11.0,2.0,0.0,1.0,0.0,0.0,71,29,749,91.9,50,2571,2.0,8,15,6,6,900,100,407,397,612,1606,19,5,266,79.9,15,1620,0.4,3,2,7,12,4,9,418,247,145,28,231,542,14,9


In [28]:
# (Recency) Goal Conversion Rate
df['Home Goal Conversion Rate'] = df['Full Time Home Goals'] / df['Home Shots on Target'].replace(0, 1)
df['Away Goal Conversion Rate'] = df['Full Time Away Goals'] / df['Away Shots on Target'].replace(0, 1)

# (Recency) Attacking Intensity
df['Home Attacking Intensity'] = df['Home Shots'] + df['Home Corners']
df['Away Attacking Intensity'] = df['Away Shots'] + df['Away Corners']

# (Recency) Disciplinary Pressure
df['Home Disciplinary Pressure'] = df['Home Fouls'] + df['Home Yellow Cards'] + df['Home Red Cards']
df['Away Disciplinary Pressure'] = df['Away Fouls'] + df['Away Yellow Cards'] + df['Away Red Cards']

df['Match Outcome'] = df['Full Time Result'].map({'H': 1, 'D': 0, 'A': -1})

ValueError: cannot reindex on an axis with duplicate labels

In [24]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df)

,Date,Home Team,Away Team,Full Time Home Goals,Full Time Away Goals,Full Time Result,Half Time Home Goals,Half Time Away Goals,Half Time Result,Referee,Home Shots,Away Shots,Home Shots on Target,Away Shots on Target,Home Corners,Away Corners,Home Fouls,Away Fouls,Home Yellow Cards,Away Yellow Cards,Home Red Cards,Away Red Cards,Home Possession,Away Possession,Home Passes Completed,Home Passes PCT,Home Progressive Passes,Home Progressive Passing Distance,Home XG,Home Take Ons Won,Home Take Ons,Home Interceptions,Home Blocks,Home Touches,Home Touches Def 3rd,Home Touches Mid 3rd,Home Touches Att 3rd,Home Carries,Home Carries Progressive Distance,Home Tackles,Home Tackles Won,Away Passes Completed,Away Passes PCT,Away Progressive Passes,Away Progressive Passing Distance,Away XG,Away Shots,Away Shots on Target REDUNDANT,Away Take Ons Won,Away Take Ons,Away Interceptions,Away Blocks,Away Touches,Away Touches Def 3rd,Away Touches Mid 3rd,Away Touches Att 3rd,Away Carries,Away Carries Progressive Distance,Away Tackles,Away Tackles Won,Home Goal Conversion Rate,Away Goal Conversion Rate,Home Attacking Intensity
0,2017-08-11,Arsenal,Leicester City,4.0,3.0,H,2.0,2.0,D,M Dean,27.0,6.0,10.0,3.0,9.0,4.0,9.0,12.0,0.0,1.0,0.0,0.0,68,32,565,83.0,62,3216,2.6,16,19,13,6,801,203,355,252,519,1686,23,17,192,61.0,16,2006,1.5,6,3,5,16,11,18,427,169,142,120,150,422,17,8,0.400000,1.000000,36.0
1,2017-08-12,Brighton & Hove Albion,Manchester City,0.0,2.0,A,0.0,0.0,D,M Oliver,6.0,14.0,2.0,4.0,3.0,10.0,6.0,9.0,0.0,2.0,0.0,0.0,23,77,146,58.9,5,1685,0.3,7,9,12,10,343,177,119,51,126,366,10,8,718,87.6,87,3285,1.9,14,4,6,12,9,9,897,130,474,298,566,1779,10,6,0.000000,0.500000,9.0
2,2017-08-12,Chelsea,Burnley,2.0,3.0,A,0.0,3.0,A,C Pawson,19.0,10.0,6.0,5.0,8.0,5.0,16.0,11.0,3.0,3.0,2.0,0.0,62,38,468,82.4,48,2448,1.5,5,8,9,6,667,201,268,203,394,1440,10,8,257,73.9,26,2521,0.6,10,5,2,3,13,13,453,168,212,74,190,460,8,4,0.333333,0.600000,27.0
3,2017-08-12,Crystal Palace,Huddersfield Town,0.0,3.0,A,0.0,2.0,A,J Moss,14.0,8.0,4.0,6.0,12.0,9.0,7.0,19.0,1.0,3.0,0.0,0.0,56,45,333,75.0,33,2847,1.1,18,23,19,9,561,180,274,113,238,751,24,19,230,64.6,18,2165,1.5,9,6,5,11,9,12,460,173,166,127,167,452,28,19,0.000000,0.500000,26.0
4,2017-08-12,Everton,Stoke City,1.0,0.0,H,1.0,0.0,H,N Swarbrick,9.0,9.0,4.0,1.0,6.0,7.0,13.0,10.0,1.0,1.0,0.0,0.0,60,40,425,78.0,32,2670,0.6,1,7,19,6,670,227,308,142,313,880,17,12,233,65.4,34,2068,0.4,9,1,10,13,24,10,473,154,192,135,190,729,18,12,0.250000,0.000000,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,2024-05-19,Crystal Palace,Aston Villa,5.0,0.0,H,2.0,0.0,H,D Bond,15.0,8.0,9.0,2.0,10.0,8.0,2.0,4.0,1.0,4.0,0.0,0.0,54,46,509,87.9,45,2499,2.5,6,12,3,12,663,122,348,199,396,969,17,13,428,85.8,11,2182,1.0,8,2,5,10,2,4,566,291,219,62,333,709,10,2,0.555556,0.000000,25.0
2756,2024-05-19,Liverpool,Wolverhampton Wanderers,2.0,0.0,H,2.0,0.0,H,C Kavanagh,36.0,4.0,14.0,3.0,14.0,11.0,10.0,2.0,1.0,1.0,0.0,1.0,67,33,592,89.2,76,3280,4.9,5,21,8,10,779,102,342,344,428,1314,22,13,250,75.3,15,1666,0.5,4,2,15,27,3,15,441,216,164,65,202,722,23,11,0.142857,0.000000,50.0
2757,2024-05-19,Luton Town,Fulham,2.0,4.0,A,1.0,2.0,A,M Donohue,15.0,15.0,6.0,7.0,15.0,20.0,4.0,4.0,5.0,4.0,0.0,0.0,41,59,265,76.6,27,2089,2.1,5,16,5,12,429,148,180,110,216,702,9,4,415,83.7,40,2849,1.1,16,8,5,12,4,9,590,235,229,129,337,650,20,9,0.333333,0.571429,30.0
2758,2024-05-19,Manchester City,West Ham United,3.0,1.0,H,2.0,1.0,H,J Brooks,28.0,3.0,12.0,2.0,3.0,12.0,11.0,2.0,0.0,1.0,0.0,0.0,71,29,749,91.9,50,2571,2.0,8,15,6,6,900,100,407,397,612,1606,19,5,266,79.9,15,1620,0.4,3,2,7,12,4,9,418,247,145,28,231,542,14,9,0.250000,0.500000,31.0
